MyAnimeList Dataset: https://www.kaggle.com/azathoth42/myanimelist 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = './drive/MyDrive/Data/anime33/'

In [ ]:
import pandas as pd
import numpy as np

anime_df = pd.read_csv('{}{}'.format(DATA_PATH, 'AnimeList.csv'))
anime_df.head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,283882,2809,Inu x Boku SS was licensed by Sentai Filmworks...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ..."
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,False,"Apr 2, 2007 to Oct 1, 2007","{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older,7.89,91206,727.0,366,204003,2579,NaN,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi..."
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,False,"Oct 4, 2008 to Sep 25, 2009","{'from': '2008-10-04', 'to': '2009-09-25'}",24 min. per ep.,PG - Children,7.55,37129,1508.0,1173,70127,802,NaN,Fall 2008,Unknown,"{'Adaptation': [{'mal_id': 101, 'type': 'manga...","TV Tokyo, Sotsu",NaN,Satelight,"Comedy, Magic, School, Shoujo","['#1: ""Minna no Tamago (みんなのたまご)"" by Shugo Cha...","['#1: ""Rottara Rottara (ロッタラ ロッタラ)"" by Buono! ..."
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,False,"Aug 16, 2002 to May 23, 2003","{'from': '2002-08-16', 'to': '2003-05-23'}",16 min. per ep.,PG-13 - Teens 13 or older,8.21,36501,307.0,916,93312,3344,Princess Tutu aired in two parts. The first pa...,Summer 2002,Fridays at Unknown,"{'Adaptation': [{'mal_id': 1581, 'type': 'mang...","Memory-Tech, GANSIS, Marvelous AQL",ADV Films,Hal Film Maker,"Comedy, Drama, Magic, Romance, Fantasy","['""Morning Grace"" by Ritsuko Okazaki']","['""Watashi No Ai Wa Chiisaikeredo"" by Ritsuko ..."
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,False,"Oct 6, 2012 to Mar 30, 2013","{'from': '2012-10-06', 'to': '2013-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,8.67,107767,50.0,426,182765,2082,NaN,Fall 2012,Unknown,"{'Adaptation': [{'mal_id': 9711, 'type': 'mang...","NHK, Shueisha",NaN,J.C.Staff,"Comedy, Drama, Romance, Shounen","['#1: ""Moshimo no Hanashi (もしもの話)"" by nano.RIP...","['#1: ""Pride on Everyday"" by Sphere (eps 1-13)..."


In [ ]:
print("anime_df: ", anime_df.shape)

anime_df:  (14478, 31)


In [ ]:
anime_df.columns

Index(['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme'],
      dtype='object')

# Basic Cosine Similarity
Results: Horrible

**features that we want to consider:**
type, episodes, rating, score, scored_by, rank, popularity, favorites, producer, studio, genre

**features to potentially consider:**
related (i.e. sequels), opening_theme, ending_theme

In [ ]:
anime_df.isnull().sum() # columns with null values

anime_id              0
title                 0
title_english      8754
title_japanese       35
title_synonyms     5541
image_url            96
type                  0
source                0
episodes              0
status                0
airing                0
aired_string          0
aired                 0
duration              0
rating                0
score                 0
scored_by             0
rank               1574
popularity            0
members               0
favorites             0
background        13421
premiered         10382
broadcast         10207
related               0
producer           6190
licensor          11105
studio             5934
genre                64
opening_theme         0
ending_theme          0
dtype: int64

In [ ]:
# cols = ['anime_id', "type", "episodes", "rating", "score", "scored_by", "rank", "popularity", "favorites", "producer", "studio", "genre"]
cols = ['anime_id', "type", "episodes", "rating", "score", "scored_by", "rank", "popularity", "favorites", "genre"]
anime = anime_df[cols]

In [ ]:
anime.shape

(14478, 10)

In [ ]:
anime.head()

,anime_id,type,episodes,rating,score,scored_by,rank,popularity,favorites,genre
0,11013,TV,12,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,2809,"Comedy, Supernatural, Romance, Shounen"
1,2104,TV,26,PG-13 - Teens 13 or older,7.89,91206,727.0,366,2579,"Comedy, Parody, Romance, School, Shounen"
2,5262,TV,51,PG - Children,7.55,37129,1508.0,1173,802,"Comedy, Magic, School, Shoujo"
3,721,TV,38,PG-13 - Teens 13 or older,8.21,36501,307.0,916,3344,"Comedy, Drama, Magic, Romance, Fantasy"
4,12365,TV,25,PG-13 - Teens 13 or older,8.67,107767,50.0,426,2082,"Comedy, Drama, Romance, Shounen"


In [ ]:
anime.isnull().sum()

anime_id         0
type             0
episodes         0
rating           0
score            0
scored_by        0
rank          1574
popularity       0
favorites        0
genre           64
dtype: int64

In [ ]:
anime = anime.dropna(axis=0) # drow null rows

In [ ]:
anime.isnull().sum()

anime_id      0
type          0
episodes      0
rating        0
score         0
scored_by     0
rank          0
popularity    0
favorites     0
genre         0
dtype: int64

In [ ]:
anime.shape  

(12848, 10)

In [ ]:
anime.head()

,anime_id,type,episodes,rating,score,scored_by,rank,popularity,favorites,genre
0,11013,TV,12,PG-13 - Teens 13 or older,7.63,139250,1274.0,231,2809,"Comedy, Supernatural, Romance, Shounen"
1,2104,TV,26,PG-13 - Teens 13 or older,7.89,91206,727.0,366,2579,"Comedy, Parody, Romance, School, Shounen"
2,5262,TV,51,PG - Children,7.55,37129,1508.0,1173,802,"Comedy, Magic, School, Shoujo"
3,721,TV,38,PG-13 - Teens 13 or older,8.21,36501,307.0,916,3344,"Comedy, Drama, Magic, Romance, Fantasy"
4,12365,TV,25,PG-13 - Teens 13 or older,8.67,107767,50.0,426,2082,"Comedy, Drama, Romance, Shounen"


In [ ]:
anime['type'].unique()

array(['TV', 'Music', 'OVA', 'ONA', 'Special', 'Movie'], dtype=object)

In [ ]:
anime['rating'].unique()

array(['PG-13 - Teens 13 or older', 'PG - Children', 'G - All Ages',
       'R+ - Mild Nudity', 'R - 17+ (violence & profanity)', 'None',
       'Rx - Hentai'], dtype=object)

In [ ]:
OHE_type = anime['type'].str.get_dummies() # one hot encoding
OHE_genre = anime['genre'].str.get_dummies(sep=',')
OHE_rating = anime['rating'].str.get_dummies()

anime.drop(['rating', 'type','genre'], axis=1, inplace=True)

# data = pd.concat([data, OHE_rating, OHE_type, OHE_genre], axis=1)
anime = pd.concat([anime, OHE_type, OHE_genre], axis=1)
anime.head()

,anime_id,episodes,score,scored_by,rank,popularity,favorites,Movie,Music,ONA,OVA,Special,TV,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,...,Thriller,Vampire,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire
0,11013,12,7.63,139250,1274.0,231,2809,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2104,26,7.89,91206,727.0,366,2579,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5262,51,7.55,37129,1508.0,1173,802,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,721,38,8.21,36501,307.0,916,3344,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,12365,25,8.67,107767,50.0,426,2082,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
anime_features = anime.iloc[:, 7:].copy()
anime_features.head()

,Movie,Music,ONA,OVA,Special,TV,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Hentai,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shoujo Ai,Shounen,Shounen Ai,Slice of Life,...,Thriller,Vampire,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,Harem,Historical,Horror,Josei,Kids,Magic,Martial Arts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,Shounen,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_sim = cosine_similarity(anime_features.values, anime_features.values)
cosine_sim

array([[1.        , 0.73029674, 0.4       , ..., 0.31622777, 0.        ,
        0.31622777],
       [0.73029674, 1.        , 0.54772256, ..., 0.28867513, 0.        ,
        0.28867513],
       [0.4       , 0.54772256, 1.        , ..., 0.31622777, 0.        ,
        0.31622777],
       ...,
       [0.31622777, 0.28867513, 0.31622777, ..., 1.        , 0.40824829,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.40824829, 1.        ,
        0.        ],
       [0.31622777, 0.28867513, 0.31622777, ..., 0.5       , 0.        ,
        1.        ]])

In [ ]:
anime_index = pd.Series(anime_df.index, index=anime_df.title).drop_duplicates()
anime_index

title
Inu x Boku SS                                0
Seto no Hanayome                             1
Shugo Chara!! Doki                           2
Princess Tutu                                3
Bakuman. 3rd Season                          4
                                         ...  
Gutchonpa Omoshiro Hanashi               14473
Geba Geba Shou Time!                     14474
Godzilla: Hoshi wo Kuu Mono              14475
Nippon Mukashibanashi: Sannen Netarou    14476
Senjou no Valkyria Special               14477
Length: 14478, dtype: int64

In [ ]:
def get_recommendation(anime_name, similarity=cosine_sim):
    idx = anime_index[anime_name]
    
    # Get the pairwsie similarity scores of all anime with that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar anime
    sim_scores = sim_scores[0:25]

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar anime
    result = anime_df[['anime_id', 'title', 'genre']].iloc[anime_indices].drop(idx)
    return result

In [ ]:
get_recommendation("Ookami to Koushinryou")

,anime_id,title,genre
415,7435,Dragon Quest: Dai no Daibouken (1991),"Adventure, Demons, Magic, Fantasy, Shounen"
1850,33138,Nama Lo Re: Namakemono The Animation,Hentai
2718,5020,Zettai Shougeki: Platonic Heart,"Action, Ecchi, Martial Arts"
2748,30427,Mr. Empty,Music
3142,3526,Mahou Shoujo Meruru,"Adventure, Fantasy, Magic, Hentai"
3448,213,Pia Carrot e Youkoso!!,"Comedy, Romance, Hentai"
4750,36423,Zhandou Wang Zhi Jufeng Zhan Hun,"Action, Kids, Shounen"
4955,6583,Super Bikkuriman,"Adventure, Fantasy, Comedy, Demons, Sci-Fi"
5259,26133,Paper Cranes Story: Kenta to Maiko,"Fantasy, Historical, Kids"
5295,20233,Tonari no Tamageta-kun,"Comedy, Sci-Fi"


# KNN & Alternating Least Squares (ALS)

## Create AnimeList Cleaned List
Create username ids and filter out NaNs

In [ ]:
user_colnames = ['username', 'anime_id', 'my_score']
user_df = pd.read_csv('{}{}'.format(DATA_PATH, 'animelists_cleaned.csv'), usecols=user_colnames)

In [ ]:
print("user_df: ", user_df.shape)
user_df.head()

user_df:  (31284030, 3)


,username,anime_id,my_score
0,karthiga,21,9
1,karthiga,59,7
2,karthiga,74,7
3,karthiga,120,7
4,karthiga,178,7


In [ ]:
anime = anime_df[['anime_id', 'title']]
anime.head()

,anime_id,title
0,11013,Inu x Boku SS
1,2104,Seto no Hanayome
2,5262,Shugo Chara!! Doki
3,721,Princess Tutu
4,12365,Bakuman. 3rd Season


In [ ]:
anime.shape

(14478, 2)

In [ ]:
print(user_df.isnull().sum())
print('shape before drop:', user_df.shape)

user_df = user_df.dropna(axis=0)

print(user_df.isnull().sum())
print('shape after drop:', user_df.shape)

username    243
anime_id      0
my_score      0
dtype: int64
shape before drop: (31284030, 3)
username    0
anime_id    0
my_score    0
dtype: int64
shape after drop: (31283787, 3)


In [ ]:
# s = pd.Series(['a', 'a','b','c', 'a'])
# labels, levels = pd.factorize(s)
# labels

user_df['username'] = user_df['username'].factorize()[0]
user_df.rename(columns= {'username': 'uid'}, inplace=True)
user_df.head()

,uid,anime_id,my_score
0,0,21,9
1,0,59,7
2,0,74,7
3,0,120,7
4,0,178,7


In [ ]:
user_df.to_csv('{}{}'.format(DATA_PATH, 'users_cleaned.csv'), index=False, header=True)

## Creating Anime-User Matrix

In [ ]:
user_df = pd.read_csv('{}{}'.format(DATA_PATH, 'users_cleaned.csv'))
user_df['my_score'] += 1
user_df.shape

(31283787, 3)

Finding the largest uid from the user_df that we can slice

In [ ]:
# user_df.iloc[25379534, :]

# id = user_df.iloc[20000000, 0]
# for i in range(20000000, 21000000):
#   if id == user_df.iloc[i, 0]:
#     print("@ idx: ", i)
#     id += 1

In [ ]:
user_df.iloc[25379530:25379534, :]

,uid,anime_id,my_score
25379532,71737,32502,1
25379533,71737,32511,11


Slice the dataset (avoid mem errors)

In [ ]:
user_df = user_df.iloc[:25379534, :]
user_df.shape

(25379534, 3)

In [ ]:
print("Number of unique users: ", len(user_df['uid'].unique()))
print("Number of anime rated by users: ", len(user_df['anime_id'].unique()))

Number of unique users:  71738
Number of anime rated by users:  6668


In [ ]:
from scipy.sparse import csr_matrix

anime_user_mat = user_df.pivot(
    index='anime_id',
    columns='uid',
    values='my_score'
).fillna(0)

anime_user_sparse = csr_matrix(anime_user_mat.values)
anime_user_mat

uid,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,71698,71699,71700,71701,71702,71703,71704,71705,71706,71707,71708,71709,71710,71711,71712,71713,71714,71715,71716,71717,71718,71719,71720,71721,71722,71723,71724,71725,71726,71727,71728,71729,71730,71731,71732,71733,71734,71735,71736,71737
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,11.0,11.0,9.0,10.0,3.0,11.0,10.0,0.0,0.0,1.0,11.0,0.0,10.0,1.0,0.0,1.0,9.0,1.0,0.0,11.0,11.0,1.0,0.0,0.0,1.0,0.0,7.0,0.0,11.0,8.0,8.0,0.0,0.0,1.0,0.0,10.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,9.0,0.0,0.0,0.0,8.0,0.0,0.0,9.0,0.0,10.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.0,11.0,0.0,8.0,9.0,8.0,9.0,9.0,0.0,0.0,0.0,11.0,0.0,1.0,0.0,0.0,1.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,11.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,10.0,10.0,8.0,10.0,9.0,11.0,9.0,0.0,0.0,1.0,9.0,0.0,9.0,0.0,0.0,0.0,8.0,1.0,0.0,1.0,1.0,11.0,0.0,0.0,11.0,0.0,4.0,0.0,1.0,0.0,11.0,1.0,0.0,11.0,0.0,10.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,10.0,0.0,0.0,8.0,0.0,0.0,10.0,0.0,0.0,0.0,7.0,0.0,9.0,0.0,10.0,1.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
7,0.0,8.0,0.0,1.0,6.0,1.0,10.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
anime_user_mat.iloc[0, 21] # (uid, anime_id)

11.0

In [ ]:
# appended_data = []

# for chunk in user_df:
#   pivot_table = chunk.pivot_table(
#       index='anime_id',
#       columns='uid',
#       values='my_score',
#       fill_value=0)
#   appended_data.append(pivot_table)
# appended_data = pd.concat(appended_data, axis=0).reset_index()